## 1. Extract data from the existing database

In [1]:
!pip install openai 
!pip install pandas
!pip install os
!pip install sqlalchemy
!pip install tqdm
!pip install string
!pip install json
!pip install python-dotenv
!pip install neo4j
!pip install timeit
!pip install time
!pip install nest_asyncio
!pip install sql
!pip3 install ipython-sql
!pip install jupysql pandas pyarrow --quiet
!pip install psycopg2-binary --quiet


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement string (from versions: none)
ERROR: No matching distribution found for string
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement timeit (from versions: none)
ERROR: No matching distribution found for timeit
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [1]:

## Load packages

import asyncio
import pandas as pd
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv
from sqlalchemy import create_engine
from tqdm import tqdm
from string import Template
import json 
from neo4j import GraphDatabase
from timeit import default_timer as timer 
from time import sleep 
import nest_asyncio
import sql
import psycopg2
from pprint import pprint
import igraph as ig
import leidenalg as la
import re
import csv

In [2]:
## OpenAI API credentials
load_dotenv()

openai_api_key= os.getenv('OPENAI_API_KEY')

## Neo4j credentials

neo4j_url = os.getenv("NEO4J_URL")
neo4j_user = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")
gds = GraphDatabase.driver(neo4j_url, auth = (neo4j_user, neo4j_password))

In [6]:
## Helper Functions
from collections import defaultdict

def process_gpt(file_prompt, system_msg):
    client = OpenAI(api_key= openai_api_key)

    completion = client.chat.completions.create(
        model = 'gpt-4o-mini',
        max_tokens = 15000,
        temperature = 0, 
        messages = [
            {"role": "system", "content": "Please output valid JSON"},
            {"role":"user", "content": file_prompt}
        ]
    )
    nlp_results = completion.choices[0].message.content
    nlp_results = nlp_results.replace('\n','')
    nlp_results = nlp_results.replace('\\n','')
    nlp_results = nlp_results.replace('\\\\','')
    reply = json.loads(nlp_results)
    return reply

## Function to take folder of files and a prompt template, and return a json-object of all the entities and relationships 

def extract_entities_relationships(content, prompt_template, doc_ids):
    start = timer()
    system_msg = "You are a helpful IT-project and account management expert who extracts information from documents."
    print(f"Running pipeline for {len(content)} files")

    results = []
    
    # Process each document individually and append its results
    for i in range(len(content)):
        try:
            print(f"Extracting entities and relationships for file number: {i}")
            text = content[i]
            prompt = Template(prompt_template).substitute(ctext=text)
            result = process_gpt(prompt, system_msg=system_msg)
            result_data = result  # Assuming result is already in dict format

            # Add the results for this document to the list
            results.append(result_data)

        except Exception as e:
            print(f"Error processing file number {i}: {e}")
            results.append(None)  # Maintain structure by appending None if error occurs
    
    # Add doc_ids to the extracted entities
    results_with_doc_ids = add_doc_id_to_entities(results, doc_ids)
    
    # Deduplicate the entities post-processing
    consolidated_results = deduplicate_entities(results_with_doc_ids)
    
    end = timer()
    print(f"Pipeline completed in {end-start} seconds")
    
    return consolidated_results

def add_doc_id_to_entities(results, doc_ids):
    """
    Add the corresponding doc_id to each entity in the result based on the document.
    """
    for i, result in enumerate(results):
        if result is None:  # Skip None values
            continue
        doc_id = doc_ids[i]  # Get the document ID
        entities = result.get("entities", [])
        for entity in entities:
            entity.setdefault("doc_ids", []).append(doc_id)  # Add doc_id to the entity
    return results

def deduplicate_entities(results):
    """
    Deduplicates entities across all documents, merging entities with the same id 
    and consolidating their doc_ids.
    """
    global_entity_tracker = defaultdict(lambda: {"entity": None, "doc_ids": []})
    global_relationships = []  # Keep track of all relationships

    for result in results:
        if result is None:
            continue
        entities = result.get("entities", [])
        relationships = result.get("relationships", [])

        # Deduplicate entities
        for entity in entities:
            entity_id = entity['id']
            
            if entity_id in global_entity_tracker:
                # If the entity already exists, combine the doc_ids
                global_entity_tracker[entity_id]["doc_ids"].extend(entity["doc_ids"])
                global_entity_tracker[entity_id]["doc_ids"] = list(set(global_entity_tracker[entity_id]["doc_ids"]))  # Remove duplicates
            else:
                # Add the new entity and its doc_ids
                global_entity_tracker[entity_id]["entity"] = entity
                global_entity_tracker[entity_id]["doc_ids"] = entity["doc_ids"]

        # Collect relationships
        global_relationships.extend(relationships)

    # Convert the global entity tracker to a list of consolidated entities
    deduplicated_entities = []
    for entity_id, entity_data in global_entity_tracker.items():
        entity = entity_data["entity"]
        entity["doc_ids"] = entity_data["doc_ids"]  # Add the consolidated list of doc_ids
        deduplicated_entities.append(entity)

    return {
        "entities": deduplicated_entities,
        "relationships": global_relationships
    }


In [5]:
df = pd.read_csv("files/updated_df.csv", lineterminator='\n')
df

,Unnamed: 0,doc_id,type,url,title,content,date_scraped
0,0,03ce3be8c9dee8c0e7abe7b6fcffa88f,webpage,https://www.lse.ac.uk/international-developmen...,Student Handbooks,\r\tStudent Handbooks\rBrowser does not suppor...,2024-09-09 12:04:45
1,1,8c2247c4ab71570d9e5e867f173d42e5,webpage,https://www.lse.ac.uk/International-Inequaliti...,Newsletter,\r\tNewsletter\rBrowser does not support scrip...,2024-09-09 11:58:42
2,2,b2acb591bf24fce689c39b1a1ad3b303,webpage,https://www.lse.ac.uk/language-centre/addition...,Additional services,\r\tAdditional services\rBrowser does not supp...,2024-09-09 11:04:31
3,3,c5190c4e0b045da868175213ac3a931f,webpage,https://www.lse.ac.uk/Economic-History/HED/Wor...,Workshops,\r\tWorkshops\rBrowser does not support script...,2024-09-09 12:44:24
4,4,67d1243e1425fd29aa4cde884afdb68d,webpage,https://www.lse.ac.uk/language-centre/undergra...,Studying at the LSE Language Centre,\r\tStudying at the LSE Language Centre\rBrows...,2024-09-09 11:00:29
...,...,...,...,...,...,...,...
22905,22905,f4f2cbf09f00bc6fa3a690867f9f2587,webpage,https://www.lse.ac.uk/africa/people/Researcher...,Ponsiano Bimeny,\r\tPonsiano Bimeny\rBrowser does not support ...,2024-09-09 16:36:02
22906,22906,2246093bc4c5214ab6f2e8b70de860c0,webpage,https://www.lse.ac.uk/ideas/people/Matthew-Benson,Matthew Benson,\r\tMatthew Benson\rBrowser does not support s...,2024-09-09 16:36:02
22907,22907,450a5fb5177759d241a9f6dc868dde9f,webpage,https://www.lse.ac.uk/international-relations/...,Jemima Ackah-Arthur,\r\tJemima Ackah-Arthur\rBrowser does not supp...,2024-09-09 16:36:03
22908,22908,3245e785414b6c1ec6d292cdc629125c,webpage,https://www.lse.ac.uk/women-peace-security/peo...,Kirsten Ainley,\r\tKirsten Ainley\rBrowser does not support s...,2024-09-09 16:36:04


## 2. Prompt engineering for entity and relations extraction 

In [13]:
#department_process_prompt is to handle academics-related entities and relationships

department_process_prompt = '''From the unstructured data provided, extract the following Entities and relationships described in the mentioned format.
0. ALWAYS FINISH THE OUTPUT. Never send partial responses.
1. First, look for these Entity types in the text and generate as comma-separated format similar to entity type.
    'id' property of each enetity must be alphanumeric and must be unique among the entities. You will be referring to this property to define the relationships between entities.
    Entity types:
    label: 'Department', id:string, name:string // Academic departments at the London School of Economics, referred to as "Department of ____"; 'id' property is the name of the department, in lowercase and camel-case.
    label: 'Course', id:string, name:string, course_code:string, prerequisites:string, semester:string // Academic courses taught at the London School of Economics; 'id' property is the name of the course, in lowercase and camel-case
    label: 'Faculty', id:string, name:string // Professors, assistant professors, teachers, fellows, tutors, directors, lecturers, and chairs associated with the London School of Economics; 'id' property is the full-name of the faculty member, in lowercase and camel-case
    label: 'Event', id:string, name:string, description:string, date:string // Seminars, workshops, talks, and other events involving the London School of Economics; 'id' property is the name of the event, in lowercase and camel-case
    label: 'Research Project', id:string, name:string, description:string // Research projects led by faculty associated with the London School of Economics; 'id' property is the name of the research project, in lowercase and camel_case


2. Next, generate each relationship as triples of head, relationship, and tail. To refer to the head and tail entity, use their respective 'id' property.
    Relationship types:
    faculty[IS_MEMBER_OF]department
    event[IS_HOSTED_BY]faculty
    event[IS_RUN_BY]department
    course[IS_RUN_BY]department
    course[IS_TAUGHT_BY]faculty
    research_project[IS_LED_BY]faculty
    research_projects[IS_ASSOCIATED_WITH]department

The output should look like:
{
    "entities": [{"label": "Department", "id": string, "name": string}],
    "relationships": ["faculty[IS_MEMBER_OF]department", "event[IS_RUN_BY]department", "course[IS_RUN_BY]department", "research_projects[IS_ASSOCIATED_WITH]department"]
}

Case Sheet:
$ctext
'''

#administration_process_prompt is to handle bureaucratic and regulatory entities and relationships

administration_process_prompt = '''From the unstructured data provided, extract the following Entities and relationships described in the mentioned format.
0. ALWAYS FINISH THE OUTPUT. Never send partial responses.
1. First, look for these Entity types in the text and generate as comma-separated format similar to entity type.
    'id' property of each entity must be alphanumeric and must be unique among the entities. You will be referring to this property to define the relationships between entities.
    Entity types:
    label: "Policy", id: string, name:string, policy_procedure_summary:string // Policy related to administrative procedure at the London School of Economics; 'id' property is the name of the policy, in lowercase and camel_case
    label: "Committee", id:string, name: string // Regulatory organisation or departments at the London School of Economics, referred to as "Department of ____"; 'id' propoerty is the name of the committee, in lowercase and camel-case
    label: "Group", id:string, name:string // Individuals or organisations; 'id' property is the name of the perosn, in lowercase and camel_case

2. Next, generate each relationship as triples of head, relationship, and tail. To refer to the head and tail entity, use their respective 'id' property.
    Relationship types:
    group[IS_AFFECTED_BY]policy
    policy[IS_ENFORCED_BY]committee
    
The output should look like:
{ "entities": [{"label": "Policy", "id": string, "name": string}],
    "relationships": ["group[IS_AFFECTED_BY]policy"]
}

Case Sheet: $ctext
''' 

#institutions_process_prompt is to handle institutions at the LSE (research facilities, summer school, careers centre, alumni centre)

#institutions_process_prompt = 
#research_process_prompt = 

full_process_prompt = '''From the unstructured data provided, extract the following Entities and relationships described in the mentioned format.
0. ALWAYS FINISH THE OUTPUT. Never send partial responses. If the input text is too long to process in one response, prioritize completing entities and relationships in the output rather than sending incomplete data.
1. First, look for these Entity types in the text and generate as comma-separated format similar to entity type.
    'id' property of each entity (EXCEPT FOR 'Document' ENTITY) must be alphanumeric and must be unique among the entities. You will be referring to this property to define the relationships between entities.
    
+ IMPORTANT: There should be no duplication of entities across documents. If an entity is found in multiple documents (e.g., "London School of Economics" or "LSE"), consolidate this into a single entity and assign it a consistent 'id' across all documents. 
    
    Entity types:
    label: 'Department', id:string, name:string, description:string, head:string, website:string, other_relevant_content:string // Academic departments at the London School of Economics, referred to as "Department of ____"; 'id' property is the name of the department, in lowercase and camelCase.
    label: 'Degree': id:string, name:string, description:string, duration:string, level:string, credits_required:string, website:string, other_relevant_content:string // Bachelor or masters university degrees that are taken at the London School of Economics, that includes "BSc Mathematics and Economics", "BA Geography", "BSc Data Science"; 'id' property is the name of the degree in lowercase and camelCase.
    label: 'Module', id:string, name:string, course_code:string, prerequisites:string, semester:string, course_content_summary:string, description:string, credit_value:string, assessment_methods:string, learning_outcomes:string, recommended_readings:string, other_relevant_content:string // Academic courses taught at the London School of Economics; 'id' property is the name of the course in lowercase and camelCase.
    label: 'Faculty', id:string, name:string, description:string, position:string, email:string, phone_number:string, office_hours:string, other_relevant_content:string // Professors, assistant professors, teachers, fellows, tutors, directors, lecturers, and chairs associated with the London School of Economics; 'id' property is the full name of the faculty member in lowercase and camelCase.
    label: 'Event', id:string, name:string, description:string, date:string, location:string, organizer:string, registration_link:string, contact_person:string, other_relevant_content:string // Seminars, workshops, talks, and other events involving the London School of Economics; 'id' property is the name of the event in lowercase and camelCase.
    label: 'ResearchProject', id:string, title:string, summary:string, description:string, start_date:string, end_date:string, principal_investigator:string, funding_amount:string, other_relevant_content:string // Research projects led by faculty associated with the London School of Economics; 'id' property is the title of the research project in lowercase and camelCase.
    label: "Policy", id:string, name:string, policy_procedure_summary:string, description:string, effective_date:string, last_reviewed_date:string, contact_person:string, related_policies:string, other_relevant_content:string // Policy related to administrative procedure at the London School of Economics; 'id' property is the name of the policy in lowercase and camelCase.
    label: "AdministrativeCommittee", id:string, name:string, description:string, chair:string, meeting_frequency:string, members:string, contact_email:string, other_relevant_content:string // Regulatory organisation or departments at the London School of Economics; 'id' property is the name of the committee in lowercase and camelCase.
    label: "Group", id:string, name:string, description:string, membership_criteria:string, number_of_members:string, leader:string, affiliated_societies:string, other_relevant_content:string // Individuals or organisations that are DIRECTLY AFFECTED BY ADMINISTRATIVE POLICIES AND PROMPTS at the London School of Economics; 'id' property is the name of the group in lowercase and camelCase.
    label: 'Publication', id:string, title:string, summary:string, description:string, publication_date:string, journal:string, doi:string, authors:string, other_relevant_content:string // A publication resulting from research conducted at LSE; 'id' property is the title of the publication in lowercase and camelCase.
    label: 'FundingSource', id:string, name:string, description:string, contact_person:string, funding_amount:string, website:string, other_relevant_content:string // An entity providing funding for research projects; 'id' property is the name of the funding source in lowercase and camelCase.
    label: 'Society', id:string, name:string, type:string, description:string, founded_year:string, current_members:string, membership_criteria:string, website:string, other_relevant_content:string // Clubs, sports clubs, and other student societies at the London School of Economics; 'id' property is the name of the society in lowercase and camelCase.
    label: 'HeadOfSociety', id:string, name:string, description:string, contact_email:string, term_start:string, term_end:string, other_relevant_content:string // Individuals leading a society, such as presidents or captains; 'id' property is the name of the individual in lowercase and camelCase.
    label: 'Facility', id:string, name:string, location:string, description:string, capacity:string, operating_hours:string, manager:string, contact_number:string, other_relevant_content:string // Facilities offered by the Student Union, such as the gym, faith center, performance venues, etc.; 'id' property is the name of the facility in lowercase and camelCase.
    label: 'Support', id:string, name:string, description:string, eligibility_criteria:string, contact_email:string, availability:string, location:string, other_relevant_content:string // Support services provided by the Student Union, such as counseling, financial aid, or advisory services; 'id' property is the name of the support service in lowercase and camelCase.
    label: 'Building', id:string, name:string, type:string, address:string, description:string, construction_year:string, floors:string, main_occupants:string, accessibility_features:string, other_relevant_content:string // Academic buildings at the London School of Economics; 'id' property is the name of the building in lowercase and camelCase.
    label: 'Office', id:string, name:string, location:string, description:string, head:string, phone_number:string, email_address:string, office_hours:string, other_relevant_content:string // Offices or administrative units within buildings; 'id' property is the name of the office in lowercase and camelCase.
    label: 'Institution', id:string, name:string, description:string, founding_year:string, director:string, affiliated_departments:string, website:string, other_relevant_content:string // Institutions affiliated with the London School of Economics that are not academic departments, including "Data Science Institute", "Eden Centre", "LSE Summer School"; 'id' property is the name of the institution in lowercase and camelCase.

2. Next, generate each relationship as triples of head, relationship, and tail. For entities that appear across multiple documents, ensure that their relationships reflect **all associated documents**.

    Relationship types:
    faculty[IS_MEMBER_OF]department
    event[IS_HOSTED_BY]faculty
    event[IS_RUN_BY]department
    module[IS_RUN_BY]department
    module[IS_TAUGHT_BY]faculty
    research_project[IS_LED_BY]faculty
    research_project[IS_ASSOCIATED_WITH]department
    degree[IS_OFFERED_BY]department
    module[IS_STUDIED_IN]degree
    module[IS_PREREQUISITE_FOR]module
    faculty[IS_IN_CHARGE_OF]module
    faculty[IS_IN_CHARGE_OF]degree
    faculty[RESIDES_IN]office
    group[IS_AFFECTED_BY]policy
    policy[IS_ENFORCED_BY]administrative_committee
    researcher[CONDUCTS]research_project
    research_project[IS_ASSOCIATED_WITH]department
    research_project[RESULTS_IN]publication
    research_project[IS_FUNDED_BY]funding_source
    publication[IS_AUTHORED_BY]researcher
    research_project[IS_GOVERNED_BY]policy
    head_of_society[LEADS]society
    society[USES]facility
    event[IS_HOSTED_BY]society
    event[TAKES_PLACE_AT]facility
    support[IS_PROVIDED_BY]facility
    department[IS_LOCATED_IN]building
    office[IS_LOCATED_IN]building
    facility[IS_PART_OF]building
    institution[IS_AFFILIATED_WITH]institution


NOTE: For the relationships template above, replace each entity label with the id of the entity in the relationship. For example, an instance of "module[IS_TAUGHT_BY]faculty" is "ds105[IS_TAUGHT_BY]profJonCardosoSilva". DO NOT write the label of the entity in the relationship.

NOTE: If a required entity or relationship is not present in the text, do not attempt to generate or infer it. Only extract and include entities and relationships that are explicitly present and verifiable in the document.

NOTE: Ensure all generated output is in a single, well-formed JSON object. Avoid any trailing commas or syntax errors. Do not include instances of "\n". The output should be a single JSON object for each document, with "entities" and "relationships" as keys.

NOTE: If an attribute is not present for a specific entity in the document, DO NOT MAKE UP INFORMATION FOR IT - LEAVE THAT ATTRIBUTE BLANK.

The output MUST be a dictionary for EACH document that you process, with each key being "entities" and "relationships". Here is a full example:
{
  "entities": [
    {
      "label": "Document",
      "id": "b2acb591bf24fce689c39b1a1ad3b303",
      "title": "Annual Report 2023",
      "type": "Report",
      "description": "Comprehensive review of activities and outcomes for the year 2023",
      "date": "2024-01-01",
      "author": "John Doe",
      "other_relevant_content": "https://www.lse.ac.uk/reports/annual2023"
    },
    {
      "label": "Policy",
      "id": "examProceduresForCandidates",
      "name": "Exam Procedures for Candidates",
      "policy_procedure_summary": "Procedures and guidelines that candidates must follow during exam periods to avoid allegations of misconduct and ensure the proper conduct of exams.",
      "description": "A policy detailing the conduct expected from candidates during exams.",
      "effective_date": "2023-09-01",
      "last_reviewed_date": "2024-01-15",
      "contact_person": "policyAdminTeam",
      "related_policies": "academicIntegrityPolicy",
      "other_relevant_content": "https://www.lse.ac.uk/policies/exam-procedures"
    },
    {
      "label": "Support",
      "id": "studentServicesCentre",
      "name": "Student Services Centre",
      "description": "Central hub for student services including registration, counseling, and support.",
      "eligibility_criteria": "All enrolled students",
      "contact_email": "ssc@lse.ac.uk",
      "availability": "Monday to Friday, 9am - 5pm",
      "location": "Old Building, Room G12",
      "other_relevant_content": "https://www.lse.ac.uk/student-services"
    },
    {
      "label": "AdministrativeCommittee",
      "id": "examsTeam",
      "name": "Exams Team",
      "description": "Committee responsible for the administration and oversight of exams.",
      "chair": "profJohnSmith",
      "meeting_frequency": "Monthly",
      "members": "profJaneDoe, profAlanWhite",
      "contact_email": "examsteam@lse.ac.uk",
      "other_relevant_content": "https://www.lse.ac.uk/exams"
    },
    {
      "label": "AdministrativeCommittee",
      "id": "studentRegulationsTeam",
      "name": "Student Regulations Team",
      "description": "Committee overseeing student conduct and regulatory compliance.",
      "chair": "profEmilyClark",
      "meeting_frequency": "Bi-weekly",
      "members": "profMichaelBrown, profSaraGreen",
      "contact_email": "regulations@lse.ac.uk",
      "other_relevant_content": "https://www.lse.ac.uk/regulations"
    },
    {
      "label": "Support",
      "id": "disabilityAndMentalHealthService",
      "name": "Disability and Mental Health Service",
      "description": "Provides support for students with disabilities and mental health needs.",
      "eligibility_criteria": "Students with registered disabilities or mental health needs",
      "contact_email": "dmh@lse.ac.uk",
      "availability": "Monday to Friday, 9am - 5pm",
      "location": "East Building, Room 3.01",
      "other_relevant_content": "https://www.lse.ac.uk/disability-mental-health"
    }
  ],
  "relationships": [
    "studentServicesCentre[IS_ASSOCIATED_WITH]examProceduresForCandidates",
    "examsTeam[IS_ASSOCIATED_WITH]examProceduresForCandidates",
    "studentRegulationsTeam[IS_ASSOCIATED_WITH]examProceduresForCandidates",
    "disabilityAndMentalHealthService[IS_ASSOCIATED_WITH]examProceduresForCandidates"
  ]
}

NOTE: Do not include instances of "\n".

Case Sheet:
$ctext
'''


In [17]:
df_content = df["content"].to_list()
df_doc_ids = df["doc_id"].to_list()

result = extract_entities_relationships(df_content, full_process_prompt, df_doc_ids)

with open("files/entities_and_relations_test.json", "w") as f:
  json.dump(result, f, indent = 4)

Running pipeline for 22910 files
Extracting entities and relationships for file number: 0


Extracting entities and relationships for file number: 1
Extracting entities and relationships for file number: 2
Extracting entities and relationships for file number: 3
Extracting entities and relationships for file number: 4
Extracting entities and relationships for file number: 5
Extracting entities and relationships for file number: 6
Extracting entities and relationships for file number: 7
Extracting entities and relationships for file number: 8
Extracting entities and relationships for file number: 9
Extracting entities and relationships for file number: 10
Extracting entities and relationships for file number: 11
Extracting entities and relationships for file number: 12
Extracting entities and relationships for file number: 13
Extracting entities and relationships for file number: 14
Extracting entities and relationships for file number: 15
Extracting entities and relationships for file number: 16
Extracting entities and relationships for file number: 17
Extracting entities and

In [13]:
with open("files/entities_and_relations_2.json", "r") as file:
    text = json.load(file)

doc_ids = df["doc_id"].to_list()
results_with_doc_id = add_doc_id_to_entities(text, doc_ids)

with open("files/entities_and_relations_2.json", "w") as f:
    json.dump(results_with_doc_id, f, indent=4)

## 3. Neo4j Cypher generation using Leiden-algorithm generated community levels

In [3]:
#Use this code if you want to test for Leiden algorithm community detection 

def create_graph_for_community_detection(entities, relationships):
    G = ig.Graph(directed=False)
    entity_ids = [entity['id'] for entity in entities]
    G.add_vertices(entity_ids)

    for rs in relationships:
        src_id, rs_type, tgt_id = rs.split("|")
        G.add_edge(src_id, tgt_id)
    
    return G

def apply_leiden_algorithm(G, entities):
    partition = la.find_partition(G, la.ModularityVertexPartition)
    for idx, entity in enumerate(entities):
        entity['community'] = partition.membership[idx]
    return entities

def generate_cypher_with_community(json_obj):
    e_statements = []
    r_statements = []
    e_label_map = {}

    for i, obj in enumerate(json_obj):
        print(f"Generating cypher for file {i+1} of {(len(json_obj))}")
        for entity in obj['entities']:
            label = entity['label']
            id = entity['id'].replace("-", "").replace("_","")
            properties = {k: v for k, v in entity.items() if k not in ['label', 'id']}
            properties['community'] = entity['community']  # Include community
            
            cypher = f"MERGE (n:{label} {{id: '{id}'}})"
            if properties:
                props_string = ', '.join([f'n.{key} = "{val}"' for key, val in properties.items()])
                cypher += f" ON CREATE SET {props_string}"
            e_statements.append(cypher)
            e_label_map[id] = label

        for rs in obj['relationships']:
            src_id, rs_type, tgt_id = rs.split("|")
            src_id = src_id.replace("-", "").replace("_","")
            tgt_id = tgt_id.replace("-", "").replace("_","")

            src_label = e_label_map[src_id]
            tgt_label = e_label_map[tgt_id]

            cypher = f'MERGE (a:{src_label} {{id: "{src_id}"}}) MERGE (b:{tgt_label} {{id: "{tgt_id}"}}) MERGE (a)-[:{rs_type}]->(b)'
            r_statements.append(cypher)
    
    with open("cyphers_with_community.txt", "w") as outfile:
        outfile.write("\n".join(e_statements + r_statements))

    return e_statements + r_statements 

# Integration of Steps in Ingestion Pipeline
def ingestion_pipeline_with_communities(folders):
    entities_relationships = []
    for key, value in folders.items():
        entities_relationships.append(extract_entities_relationships(key, value))

    for obj in entities_relationships:
        entities = obj['entities']
        relationships = obj['relationships']

        # Create the graph and apply Leiden algorithm
        G = create_graph_for_community_detection(entities, relationships)
        entities_with_communities = apply_leiden_algorithm(G, entities)

        # Replace entities with annotated ones
        obj['entities'] = entities_with_communities

    # Generate and execute Cypher queries
    cypher_statements = generate_cypher_with_community(entities_relationships)
    for i, stat in enumerate(cypher_statements):
        print(f"Executing cypher statement {i+1} of {len(cypher_statements)}")
        try:
            gd.execute_query(stat)
        except Exception as e:
            with open("failed_statements.txt", "a") as f:
                f.write(f"{stat} - Exception: {e}\n")

with open("files/entities_and_relations_2.json", "r") as f:
    json_obj = json.load(f)

cypher = generate_cypher(json_obj)


NameError: name 'generate_cypher' is not defined

## 4. Neo4j Cypher Generation without community levels

In [5]:
# Function takes json-obejcts of entities and relationships and generates a cypher query for creating those entities
# Does so without use of Leiden clustering

import re

def generate_cypher(json_obj):
    e_statements = []
    r_statements = []
    e_label_map = {}

    # Loop through our json object
    for i, obj in enumerate(json_obj):
        print(f"Generating cypher for file {i+1} of {len(json_obj)}")

        # Skip if obj is None
        if obj is None:
            print(f"Skipping null object at index {i}")
            continue
        
        # Process entities
        for entity in obj.get("entities", []):
            # Skip if entity is None
            if entity is None:
                continue
            label = entity.get("label")
            id = entity.get("id")
            
            # Skip if label or id is None
            if label is None or id is None:
                continue
            
            id = id.replace("-", "").replace("_", "")
            properties = {k: v for k, v in entity.items() if k not in ["label", "id"]}

            cypher = f'MERGE (n:{label} {{id: "{id}"}})'
            if properties:
                props_str = ", ".join([f'n.{key} = "{val}"' for key, val in properties.items()])
                cypher += f" ON CREATE SET {props_str}"
            
            e_statements.append(cypher)
            e_label_map[id] = label  # Store the label in the map

        # Process relationships
        for rs in obj.get("relationships", []):
            # Skip if relationship is None
            if rs is None:
                continue
            
            # Use regular expression to extract src_id, relationship type, and tgt_id
            match = re.match(r"(.+)\[(.+)\](.+)", rs)
            if not match:
                print(f"Skipping invalid relationship format: {rs}")
                continue

            # Extract the components from the regex match
            src_id, rs_type, tgt_id = match.groups()
            
            # Clean the IDs
            src_id = src_id.replace("-", "").replace("_", "")
            tgt_id = tgt_id.replace("-", "").replace("_", "")

            # Get the source and target labels from the map
            src_label = e_label_map.get(src_id)
            tgt_label = e_label_map.get(tgt_id)

            # Skip if either source or target label is missing
            if not src_label or not tgt_label:
                print(f"Skipping relationship due to missing entity label for {src_id} or {tgt_id}")
                continue

            # Generate the relationship cypher statement
            cypher = f'MERGE (a:{src_label} {{id: "{src_id}"}}) MERGE (b:{tgt_label} {{id: "{tgt_id}"}}) MERGE (a)-[:{rs_type}]->(b)'
            r_statements.append(cypher)

    return e_statements + r_statements

    # Write the generated cypher statements to a file
    with open("cyphers.txt", "w") as outfile:
        outfile.write("\n".join(e_statements + r_statements))

    return e_statements + r_statements

def ingestion_pipeline(cypher_statements):
    for i, stmt in enumerate(cypher_statements):
        print(f"Executing cypher statement {i+1} of {len(cypher_statements)}")
        try:
            gds.execute_query(stmt)
        except Exception as e:
            # Log the failed statement to a file
            with open("files/failed_statements.txt", "a") as f:  # Use 'a' to append to the file instead of overwriting it
                f.write(f"{stmt} - Exception: {e}\n")

with open("files/entities_and_relations_2.json", "r") as f:
    json_obj = json.load(f)

cypher = generate_cypher(json_obj)

Generating cypher for file 1 of 4106
Generating cypher for file 2 of 4106
Generating cypher for file 3 of 4106
Generating cypher for file 4 of 4106
Generating cypher for file 5 of 4106
Generating cypher for file 6 of 4106
Generating cypher for file 7 of 4106
Generating cypher for file 8 of 4106
Generating cypher for file 9 of 4106
Generating cypher for file 10 of 4106
Generating cypher for file 11 of 4106
Generating cypher for file 12 of 4106
Generating cypher for file 13 of 4106
Generating cypher for file 14 of 4106
Generating cypher for file 15 of 4106
Skipping relationship due to missing entity label for urbanGrowthSeminar or Daniel M. Sturm
Generating cypher for file 16 of 4106
Generating cypher for file 17 of 4106
Generating cypher for file 18 of 4106
Generating cypher for file 19 of 4106
Generating cypher for file 20 of 4106
Generating cypher for file 21 of 4106
Generating cypher for file 22 of 4106
Generating cypher for file 23 of 4106
Generating cypher for file 24 of 4106
Skipp

In [6]:
cypher

['MERGE (n:Publication {id: "predictingOurClimateFuture"}) ON CREATE SET n.title = "Predicting Our Climate Future: What We Know, What We Don’t Know, and What We Can’t Know", n.summary = "A book that argues for a re-evaluation of how we study climate change across various disciplines.", n.description = "This book addresses fundamental questions about climate change and the limitations of current scientific understanding.", n.publication_date = "2023", n.journal = "Oxford University Press", n.doi = "", n.authors = "David Stainforth", n.other_relevant_content = "", n.doc_id = "00019fdfa72ca963b5d236470f52e2e4"',
 'MERGE (n:Faculty {id: "davidStainforth"}) ON CREATE SET n.name = "David Stainforth", n.description = "Professorial Research Fellow in the Grantham Research Institute at LSE and Honorary Professor in the Physics Department at Warwick University.", n.position = "Professorial Research Fellow", n.email = "", n.phone_number = "", n.office_hours = "", n.other_relevant_content = "Resea

In [8]:
ingestion_pipeline(cypher)

Executing cypher statement 1 of 36488


Unable to retrieve routing information
Transaction failed and will be retried in 1.0020138979155706s (Unable to retrieve routing information)


Unable to retrieve routing information
Transaction failed and will be retried in 2.2012256945116238s (Unable to retrieve routing information)


Executing cypher statement 2 of 36488
Executing cypher statement 3 of 36488
Executing cypher statement 4 of 36488
Executing cypher statement 5 of 36488
Executing cypher statement 6 of 36488
Executing cypher statement 7 of 36488
Executing cypher statement 8 of 36488
Executing cypher statement 9 of 36488
Executing cypher statement 10 of 36488
Executing cypher statement 11 of 36488
Executing cypher statement 12 of 36488
Executing cypher statement 13 of 36488
Executing cypher statement 14 of 36488
Executing cypher statement 15 of 36488
Executing cypher statement 16 of 36488
Executing cypher statement 17 of 36488
Executing cypher statement 18 of 36488
Executing cypher statement 19 of 36488
Executing cypher statement 20 of 36488
Executing cypher statement 21 of 36488
Executing cypher statement 22 of 36488
Executing cypher statement 23 of 36488
Executing cypher statement 24 of 36488
Executing cypher statement 25 of 36488
Executing cypher statement 26 of 36488
Executing cypher statement 27 of 